## Essa é minha solução para o desafio técnico do processo seleção de estágio da Solvimm.

### Nome: Raphael Balmant Rodrigues Chaves


### [Link para o desafio](https://drive.google.com/file/d/1dYdoacGbVmQ8JyPjpxCkCSuR3FGTDB3n/view)

#### Arquivos necessários: (não será necessário fazer download manual deles, o programa fará isso!)

*   [movies.csv](https://drive.google.com/file/d/1gLsCjaMrL91ECdThq58cZAzB9tPxG18g/view) (id: 1gLsCjaMrL91ECdThq58cZAzB9tPxG18g)
*   [customers_rating.csv](https://drive.google.com/file/d/1C_T1w8fc7Oa8MeTo4LMTEcv90IfEOS-6/view) (id: 1C_T1w8fc7Oa8MeTo4LMTEcv90IfEOS-6)

#### Documentação utilizada:

*   [wkentaro/gdown: Download a large file from Google Drive  (curl/wget fails because of the security notice).](https://github.com/wkentaro/gdown) (para fazer download dos arquivos necessários)

*   [Comparison with SQL &#8212; pandas 1.3.1 documentation](https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_sql.html) (esse link me ajudou **bastante** a entender como o pandas funciona, a partir de comparações com SQL)

*   [pandas.read_csv &#8212; pandas 1.3.1 documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) (para entender como gerar DataFrame a partir dos arquivos)

*   [pandas.DataFrame.groupby &#8212; pandas 1.3.1 documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) (utilizei para entender melhor como a função groupby funciona)

*   [pandas.core.groupby.GroupBy.size &#8212; pandas 1.3.1 documentation](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.size.html) (para entender o size depois do groupby)

*   [pandas.DataFrame.nsmallest &#8212; pandas 1.3.1 documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nsmallest.html) (para conseguir pegar menores valores em entradas em DataFrame, baseado em alguma coluna)

*   [pandas.DataFrame.nlargest &#8212; pandas 1.3.1 documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nlargest.html) (utilidade semelhante ao link acima, mas para maiores valores)

*   [pandas.DataFrame.reset_index &#8212; pandas 1.3.1 documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html) (utilizei para adicionar coluna de contagem depois de agrupar um DataFrame por alguma coluna)

#### Formatos:

> movies.csv - Movie_Id;(Title, Year) (formato atual)

> movies.csv - Movie_Id;Title;Year **(formato desejado)**

> customers_rating.csv - Cust_Id;Rating;Date;Movie_Id

### Sobre o dataset acima, somos perguntados:

1.   Quantos filmes estão disponíveis no dataset?
2.   Qual é o nome dos 5 filmes com melhor média de avaliação?
3.   Quais os 5 anos com menos lançamentos de filmes?
4.   Quantos filmes que possuem avaliação maior ou igual a 4.7, considerando apenas os filmes avaliados na última data de avaliação do dataset?
5.   Dos filmes encontrados na questão anterior, quais são os 10 filmes com as piores notas e quais as notas?
6.   Quais os id's dos 5 customer que mais avaliaram filmes e quantas avaliações cada um fez?

Bom, antes de sequer começar a pensar nas perguntas, precisamos primeiro obter os arquivos do dataset. Para isso, vamos aqui utilizar a biblioteca gdown, assim vamos conseguir os arquivos diretamente sem ter que fazer upload para o nosso próprio google drive. Para garantir que o gdown (e o numpy e pandas) estão instalados, por favor, rode a seguinte caixa de código:





In [106]:
!pip install gdown
!pip install numpy
!pip install pandas

Agora com o gdown instalado (garantidamente), podemos obter os arquivos do dataset através dos IDs presentes nos links fornecidos no PDF do desafio. Dessa forma, para salvar o arquivo do google drive movies.csv, execute o seguinte comando: 

In [107]:
!gdown -O movies.csv --id 1gLsCjaMrL91ECdThq58cZAzB9tPxG18g 

Downloading...
From: https://drive.google.com/uc?id=1gLsCjaMrL91ECdThq58cZAzB9tPxG18g
To: /content/movies.csv
100% 160k/160k [00:00<00:00, 44.1MB/s]


E agora o arquivo customers_rating.csv o comando:

In [108]:
!gdown -O customers_rating.csv --id 1C_T1w8fc7Oa8MeTo4LMTEcv90IfEOS-6

Downloading...
From: https://drive.google.com/uc?id=1C_T1w8fc7Oa8MeTo4LMTEcv90IfEOS-6
To: /content/customers_rating.csv
658MB [00:05, 129MB/s]


Temos os arquivos, mas note que o arquivo movies.csv tem um problema grave de formatação dos dados, eles estão no formato: [id];([título], [ano]) que não é ideal para carregarmos com o pandas ainda. Vamos então deixar o arquivo em um formato amigável para o pandas :)

In [109]:
# Vamos abrir o arquivo para obter seu conteúdo. (a função open faz isso pra gente)
with open('movies.csv', 'r+') as fmovies:
  content = fmovies.readlines()

# Agora podemos operar o conteúdo do arquivo, e fazer as alterações necessárias.

lines = []

for line in content:
  # Formato atual: [id];([titulo], [ano])
  # Formato pretendido: [id];[titulo];[ano]

  id = ""
  title = ""
  year = ""

  # Flags para onde estamos na linha
  id_ok = False
  title_ok = False
  year_ok = False

  i = 0
  while i < len(line):
    if line[i] == ';':
      id_ok = True
      i = i + 2 # Pulamos o caractere '('
      continue

    if line[i] == ',' and line[i+1] == ' ' and line[i+2].isdigit():
      title_ok = True
      i = i + 2 # Pulamos o espaço (posição i+1)
      continue

    if line[i] == ')':
      year_ok = True
      break # Acabamos de ler os dados da linha.

    if id_ok and title_ok:
      year = year + line[i]
    elif id_ok:
      title = title + line[i]
    else:
      id = id + line[i]

    i = i + 1

  # Para a linha atual, temos os dados (id, title, year)
  # Vamos agora colocar no formato desejado

  line = id + ';' + title + ';' + year
  lines.append(line)

# Agora podemos salvar os dados processados de volta no arquivo. (opção 'w')
with open('movies.csv', 'w') as fmovies:
  for line in lines:
    fmovies.write("%s\n" % line)



Agora que estamos em posse de todos os arquivos necessários, precisamos antes de tudo importar a biblioteca pandas, que iremos utilizar para analisar o dataset e sermos capazes de responder as perguntas, e vamos também importar a numpy, caso seja necessário utilizarmos. E podemos também carregar os dois arquivos CSV. A função read_csv do pandas é responsável por carregar o arquivo CSV (agora arquivo local) em memória no formato DataFrame do pandas.

In [110]:
import pandas as pd
import numpy as np

# O arquivo movies.csv está com dados separados por ';', e não contém cabeçalho na primeira linha. (por isso, vamos inserior nomes para as nossas colunas)
movies = pd.read_csv('movies.csv', delimiter=';', names=['Movie_Id', 'Title', 'Year'])

# Já o arquivo customers_rating.csv está também com dados separaos por ';', mas contém cabeçalho na primeira linha.
customers_rating = pd.read_csv('customers_rating.csv', delimiter=';')

**1. Quantos filmes estão disponíveis no dataset?**

Para descobrir quantos filmes temos no dataset, assumindo que existe consistência entre os dois arquivos, e também a pergunta é sobre os filmes **disponíveis**, então basta pegarmos a quantidade de linhas (entradas) em movies.csv.

In [111]:
print("1. Temos %d filmes disponíveis no dataset." % (len(movies.index)))

1. Temos 4499 filmes disponíveis no dataset.


**2. Qual é o nome dos 5 filmes com melhor média de avaliação?**

Primeiramente, vamos ter que agrupar as avaliações por filme (Movie_Id), e então tomar a média da coluna Rating, com isso vamos então fazer um join (merge do pandas é o equivalente ao join do SQL) para pegar o nome do filme, e por fim vamos contar os 5 que tem melhores médias. 

In [121]:
# Pegamos os filmes com suas médicas de avaliação.
rows = customers_rating.groupby(by="Movie_Id").agg({"Rating": np.mean})

# Right join com movies para pegar os nomes do filmes.
rows = pd.merge(movies, rows, on='Movie_Id', how='right')

# Estamos interessados apenas nos filmes com 5 maiores médias.
rows = rows.nlargest(5, columns=['Rating'])

print("2. Os nomes dos 5 filmes com melhor média de avaliação são os seguintes:\n")

print(rows['Title'])


2. Os nomes dos 5 filmes com melhor média de avaliação são os seguintes:

3455                                       Lost: Season 1
3032     Ghost in the Shell: Stand Alone Complex: 2nd Gig
2101                               The Simpsons: Season 6
4237                                            Inu-Yasha
12      Lord of the Rings: The Return of the King: Ext...
Name: Title, dtype: object


**3. Quais os 5 anos com menos lançamentos de filmes?**

Primeiro devemos agrupar a listagem de filmes por ano de lançamento, fazer a contagem respectiva de cada ano, e então ver quais ano tem maior soma de filmes.

In [113]:
# Agrupamos por nome e pegamos a quantidade por ano. (veja que adicionamos uma nova coluna 'Count' com a contagem de títulos naquele ano)
rows = movies.groupby(by="Year").size().reset_index(name='Count')

# Estamos interessados nos 5 anos com *menor* número de filmes
rows = rows.nsmallest(5, columns=['Count'])

print("3. Os 5 anos com menor número de lançamentos de filmes foram:\n")

# Queremos apenas os anos
print(rows['Year'])

3. Os 5 anos com menor número de lançamentos de filmes foram:

0    1915.0
2    1917.0
6    1922.0
1    1916.0
3    1918.0
Name: Year, dtype: float64


**4. Quantos filmes que possuem avaliação maior ou igual a 4.7, considerando apenas os filmes avaliados na última data de avaliação do dataset?**

Precisamos primeiro filtrar os dados pela última data de avaliação, e depois selecionar, dentre esses, aqueles que tem média de avaliação maior ou igual a 4.7

In [114]:
# Última data de avaliação no dataset.
last_date = customers_rating['Date'].max()

# Entradas da última data de avaliação do dataset.
rows = customers_rating[customers_rating['Date'] == last_date]

# Toma a média de todas as avaliações, agrupando por ID do filme.
rows = rows.groupby(by="Movie_Id").agg({"Rating": np.mean})

# Seleciona apenas entradas com médica de avaliações maior ou igual a 4.7
rows = rows[rows['Rating'] >= 4.7]

# Printa quantas entradas temos em rows, ou seja, quantos filmes tiveram avaliação maior ou igual a 4.7 no último dia de avaliação.
print("4. %d filmes tiveram avaliação maior ou igual a 4.7 no dia %s (último dia de avaliações no dataset)" % (rows.count(), last_date))

4. 195 filmes tiveram avaliação maior ou igual a 4.7 no dia 2005-12-31 (último dia de avaliações no dataset)


**5. Dos filmes encontrados na questão anterior, quais são os 10 filmes com as piores notas e quais as notas?**

Os filmes da questão anterior ainda estão na variável rows, então podemos simplesmente partir daí. Mais uma vez vamos ter que fazer um join com movies, para obter quais os filmes correspondentes aos Movie_Id encontrados na questão anterior.

In [115]:
# Left join com mocies para pegar o nome dos títulos.
rows = pd.merge(movies, rows, on='Movie_Id', how='right')

# Selecionamos a 10 piores médias de avaliação dentre os filmes da questão anterior.
rows = rows.nsmallest(10, columns=['Rating'])

print("5. Dos filmes encontrados na questão anterior, os 10 filmes com piores médias (e suas respectivas médias) são:\n")

# Só queremos o título e a média de avaliação (nota)
print(rows[['Title', 'Rating']])

5. Dos filmes encontrados na questão anterior, os 10 filmes com piores médias (e suas respectivas médias) são:

                               Title    Rating
150                    Spirited Away  4.714286
59        The Twilight Zone: Vol. 15  4.750000
89       An Evening With Kevin Smith  4.750000
94           Gilmore Girls: Season 3  4.750000
102       The Twilight Zone: Vol. 41  4.750000
126    Absolutely Fabulous: Series 2  4.750000
161                    Pete's Dragon  4.750000
192  SpongeBob SquarePants: Season 3  4.750000
33              In the Mood for Love  4.800000
187   Curb Your Enthusiasm: Season 3  4.800000


**6.  Quais os id's dos 5 customer que mais avaliaram filmes e quantas avaliações cada um fez?**

Precisamos agrupar as avaliações por Cust_Id, e contar quantas avaliações temos por Cust_Id, de maneira semelhante ao que fizemos na questão 3.

In [116]:
# De maneira semelhante a questão 3, agrupamos as entradas de customers_rating por Cust_Id, e fazemos uma contagem adicionando essa contagem em uma coluna 'N_Ratings' 
rows = customers_rating.groupby(by="Cust_Id").size().reset_index(name='N_Ratings')

# Queremos encontrar os consumidores que mais avaliaram, ou seja, que tiveram maior soma no comando anterior. (coluna 'N_Ratings')
rows = rows.nlargest(5, columns=['N_Ratings'])

print("6. Os ids dos 5 customers que mais avaliaram filmes (com a respectiva quantidade de avaliações) são:\n")

print(rows)

6. Os ids dos 5 customers que mais avaliaram filmes (com a respectiva quantidade de avaliações) são:

        Cust_Id  N_Ratings
54307    305344       4467
69083    387418       4422
433441  2439493       4195
295928  1664010       4019
376450  2118461       3769
